In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from category_encoders.one_hot import OneHotEncoder 
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier 

#### Importing the data

In [2]:
# Importing the train data
train = pd.read_csv(r'../input/spaceship-titanic/train.csv',sep=',')
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
# Importing the test data
test = pd.read_csv(r'../input/spaceship-titanic/test.csv',sep=',')
test.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


#### Cleaning the Data

In [4]:
train.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [5]:
train.fillna(0,inplace = True)

In the 'Cabin' axis there are 3 'hidden' variables. Let's split them to perform the transformation to numeric features later.

In [6]:
train[['Deck','Num','Side']]= train['Cabin'].str.split(pat='/',expand=True)

In [7]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported,Deck,Num,Side
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False,B,0,P
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True,F,0,S
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False,A,0,S
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False,A,0,S
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True,F,1,S


In [8]:
train.dtypes

PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
Deck             object
Num              object
Side             object
dtype: object

In [9]:
train.fillna(0,inplace=True)

In [10]:
train['Num'] = train['Num'].astype(int)

Making a One Hot Encoder with categorical features

In [11]:
one_hot_encoder = OneHotEncoder(cols=['HomePlanet','CryoSleep','Destination','VIP','Deck','Side'])

In [12]:
train = one_hot_encoder.fit_transform(train)

Excluding the categorical data

In [13]:
train.drop(['PassengerId','Name','Cabin'],inplace = True,axis=1)

#### Spliting the data

Initially, we will split the data into training and testing. For the data below, I'll split it as 80/20.

In [14]:
y = train['Transported']
X = train.drop(['Transported'],axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#### Random Forest Classifier

First, we'll use Random Forest to see how it will perform.

In [15]:
clf = RandomForestClassifier(max_depth=2,random_state=0)

In [16]:
clf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

Random Forest performed well, let's test ADABoost now to see how it performs.

In [17]:
predict_RF = clf.predict(X_val)

accuracy_score(y_val, predict_RF)

0.7423806785508913

#### ADABoosting Classifier

In [18]:
regr = AdaBoostClassifier(random_state=42, n_estimators=100)

In [19]:
regr.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [20]:
predict_ADA = regr.predict(X_val)

In [21]:
accuracy_score(y_val, predict_ADA)

0.7832087406555491

Finally, let's test Gradient Boosting to see if it will be more accurate.

#### Gradient Boosting Classifier

In [22]:
GBC = GradientBoostingClassifier(n_estimators = 100,random_state=42,max_depth=2,learning_rate=1.0,max_features=2)


In [23]:
GBC.fit(X_train, y_train)

GradientBoostingClassifier(learning_rate=1.0, max_depth=2, max_features=2,
                           random_state=42)

In [24]:
predict_gbc = GBC.predict(X_val)

In [25]:
accuracy_score(y_val, predict_gbc)

0.7901092581943646

#### Conclusion


Finally, for the data in question we had a better performance using Gradient Boosting. However, we also obtained satisfactory results with Random Forest and ADABoosting.

In [26]:
print('Random Forest:',accuracy_score(y_val, predict_RF))
print('ADABoost:',accuracy_score(y_val, predict_ADA))
print('Gradient Boosting Classifier:',accuracy_score(y_val, predict_gbc))

Random Forest: 0.7423806785508913
ADABoost: 0.7832087406555491
Gradient Boosting Classifier: 0.7901092581943646
